In [2]:
# The following taken from 2021 notebook

# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math
import random
from collections import Counter
import statistics
import parse
import operator
from functools import reduce

# aliases from utils
getnums = aoc_utils.getnums

# from norvig's pytudes
cat = ''.join

# Helper Functions

In [3]:
import functools

def toIntList(text, sep=","):
    if sep:
        return list(map(int, text.split(sep)))
    else:
        return list(map(int, text))

assert toIntList("1,2,3") == [1,2,3]
assert toIntList("123", sep=None) == [1,2,3]

def readInput(dayNum):
    return open("./data/day{}-input.txt".format(dayNum), "r").read().strip()

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

assert gcd(2, 10) == 2
assert gcd(22, 11) == 11
assert gcd(23, 24) == 1

def lcm(a, b):
    return a * b // gcd(a,b)

assert lcm(2,11) == 22
assert lcm(3, 8) == 24
assert lcm(5, 15) == 15
assert lcm(15, 5) == 15

def lcmm(*args):
    return functools.reduce(lcm, args)

assert lcmm(100, 23, 98) == 112700
assert lcmm(18, 28, 44) == 2772


def gridNeighbors(grid, pos):
    """Given a grid and a (x,y) pos, returns the neighbors at the cardinal (N,E,S,W) directions"""
    (x,y) = pos
    movements = [
        (-1,0),
        (0,-1),
        (1,0),
        (0,1)
    ]
    return [ (x+dX,y+dY) for (dX,dY) in movements]     

def gridPointsAtDistance(point, dimensions, distance):
    """
        Given a (x,y) point, and the (width,height) dimensions and a distance,
        returns all the surrounding cells in the grid at that distance away
    """
    (x0, y0) = point
    (width, height) = dimensions
    points = set()
    for x in range(max(0, x0 - distance), min(x0 + distance + 1, width)):
        if x < 0 or x > width:
            continue
        y = y0 - distance
        if y >= 0:
            points.add((x, y))
        y = y0 + distance
        if y < height:
            points.add((x, y))
    for y in range(max(0, y0 - distance), min(y0 + distance + 1, height)):
        if y < 0 or y > height:
            continue
        x = x0 - distance
        if x >= 0:
            points.add((x,y))
        x = x0 + distance
        if x < width:
            points.add((x,y))
    return points

assert gridPointsAtDistance( (0,0), (4,4), 1) == set([
           (0,1),
    (1,0), (1,1),
])
assert gridPointsAtDistance( (1,1), (4,4), 1) == set([
    (0,0),  (0,1), (0,2),
    (1,0),         (1,2),
    (2,0),  (2,1), (2,2)
])
assert gridPointsAtDistance( (1,1), (3,3), 1) == set([
    (0,0),  (0,1), (0,2),
    (1,0),         (1,2),
    (2,0),  (2,1), (2,2)
])
assert gridPointsAtDistance( (0,0), (3,3), 2) == set([
                   (0,2),
                   (1,2),
    (2,0),  (2,1), (2,2)
])
assert gridPointsAtDistance( (1,1), (4,4), 2) == set([
                          (0,3),
                          (1,3),
                          (2,3),
    (3,0),  (3,1), (3,2), (3,3),
])

# Day 1

In [95]:
class Day1:
    def solve(self, text):
        data = map(int, text.split("\n"))
        return sum([x//3-2 for x in map(int, data)])

assert Day1().solve(readInput(1)) == 3256794

print(Day1().solve(readInput(1)))

3256794


In [96]:
def neededFuel(mass):
    if mass <= 0:
        return 0
    needed = mass // 3 - 2
    if needed <= 0:
        return 0
    else:
        return needed + neededFuel(needed)

In [97]:
assert sum([neededFuel(x) for x in map(int, readInput(1).split('\n'))]) == 4882337

# [Day 2](https://adventofcode.com/2019/day/2)


In [98]:
text = readInput(2)

In [99]:
class IntCode:
    OPS = {1: 'add', 2: 'mult', 99: 'halt'}
    ADVANCE_COUNT = {
        1: 4,
        2: 4,
        99: 0
    }

    def __init__(self, mem=[]):
        self.mem = mem
        self.ip = 0
        self.done = False
        
    def getOperand(self, ipOffset):
        return self.mem[self.getAddr(ipOffset)]
    
    def getAddr(self, ipOffset):
        return self.mem[self.ip + ipOffset]
    
    def add(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getAddr(3)
        self.mem[destAddr] = operands[0] + operands[1]

    def mult(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getAddr(3)
        self.mem[destAddr] = operands[0] * operands[1]
        
    def unknown(self):
        print("Unknown opcode {}".format(self.opcode()))
        raise
        
    def halt(self):
        self.done = True
    
    def opcode(self):
        return self.mem[self.ip]
    
    def op(self):
        op = getattr(self, IntCode.OPS.get(self.opcode(), 'unknown'))
        op()
        self.advance()
        
    def advance(self):
        self.ip += IntCode.ADVANCE_COUNT[self.opcode()]
    
    def getOperands(self):
        count = 0
        
    def run(self):
        while not self.done:
            self.op()
        return self.mem[0]
    
class Day2:
    def solve(self, text):
        mem = toIntList(text)
        mem[1] = 12
        mem[2] = 2
        return IntCode(mem).run()

    def solveB(self, text):
        target = 19690720

        for noun in range(100):
            for verb in range(100):
                try:
                    mem = toIntList(text)
                    mem[1] = noun
                    mem[2] = verb
                    result = IntCode(mem).run()
                    if result == target:
                        return noun*100+verb
                except:
                    next
            
        
assert Day2().solve(text) == 6087827
assert Day2().solveB(text) == 5379
print(Day2().solve(text), Day2().solveB(text))

6087827 5379


# [Day 3](https://adventofcode.com/2019/day/3)

In [100]:
text = """R1001,D915,R511,D336,L647,D844,R97,D579,L336,U536,L645,D448,R915,D473,L742,D470,R230,D558,R214,D463,L374,D450,R68,U625,L937,D135,L860,U406,L526,U555,R842,D988,R819,U995,R585,U218,L516,D756,L438,U921,R144,D62,R238,U144,R286,U934,L682,U13,L287,D588,L880,U630,L882,D892,R559,D696,L329,D872,L946,U219,R593,U536,R402,U946,L866,U690,L341,U729,R84,U997,L579,D609,R407,D846,R225,U953,R590,U79,R590,U725,L890,D384,L442,D364,R600,D114,R39,D962,R413,U698,R762,U520,L180,D557,R35,U902,L476,U95,R830,U858,L312,U879,L85,U620,R505,U248,L341,U81,L323,U296,L53,U532,R963,D30,L380,D60,L590,U699,R967,U88,L725,D730,R706,D337,L248,D46,R131,U541,L313,U508,R120,D719,R28,U342,R555,U780,R397,D523,L619,D820,R865,D4,L790,D544,L873,D249,L220,U343,R818,U803,R309,D576,R811,D717,L800,D171,R523,U630,L854,U265,R207,U147,R518,U237,R822,D672,L140,U580,R408,D739,L519,U759,R664,D61,R258,D313,R472,U437,R975,U828,L54,D892,L370,U509,L80,U593,L268,U856,L177,U950,L266,U29,R493,D228,L110,U390,L92,U8,L288,U732,R459,D422,R287,D359,R915,U295,R959,U215,R82,D357,L970,D782,L653,U399,L50,D720,R788,D396,L562,D560,R798,D196,R79,D732,R332,D957,L106,D199,R756,U379,R716,U282,R812,U346,R592,D416,L454,U612,L160,U884,R373,U306,R55,D492,R175,D233,L249,D616,L342,D650,L181,U868,L761,D170,L976,U711,R377,D113,L548,U39,R62,D99,R853,U249,L951,U617,R257,U457,R430,D355,L541,U595,L176,D987,R365,D77,L181,D192,L688,D942,R617,U484,R247,U180,R771,D392,R184,U597,L682,U454,R856,U616,R174,U629,L607,U41,L970,D602,R402,D208,R826
L994,U238,R605,U233,L509,U81,R907,U880,R666,D86,R6,U249,R345,D492,L912,U770,L827,D107,R988,D525,L471,U706,R31,U485,R835,D778,R419,D461,L937,D740,R559,U309,L379,U385,R828,D698,R276,U914,L911,U969,R282,D365,L43,D911,R256,D592,L451,U162,L829,D564,R349,U279,R19,D110,R259,D551,L172,D899,L924,D819,R532,U737,L794,U995,R168,D359,R847,U426,R224,U984,L929,D531,L797,U292,L332,D280,R317,D648,R776,D52,R916,U363,R919,U890,R583,U961,L89,D680,L894,D226,L83,U68,R551,U413,R259,D468,L702,U453,L128,U986,R238,U805,R431,U546,R944,D142,R677,D783,R336,D220,R40,U391,R5,D760,L963,D764,R653,U932,R473,U311,L189,D883,R216,U391,L634,U275,L691,U975,R130,D543,L163,U736,R964,U729,R752,D531,R90,D471,R687,D341,R441,U562,R570,U278,R570,U177,L232,U781,L874,U258,R180,D28,R916,D395,R96,U954,L222,U578,L394,U775,L851,D18,L681,D912,L761,U945,L866,D12,R420,D168,R490,U679,R521,D91,L782,U583,L823,U656,L365,D517,R319,U725,L824,D531,L747,U822,R893,D162,L11,D913,L295,D65,L393,D351,L432,U828,L131,D384,R311,U381,L26,D635,L180,D395,L576,D836,R548,D820,L219,U749,L64,D2,L992,U104,L501,U247,R693,D862,R16,U346,R332,U618,R387,U4,L206,U943,R734,D164,R771,U17,L511,D475,L75,U965,R116,D627,R243,D77,R765,D831,L51,U879,R207,D500,R289,U749,R206,D850,R832,U407,L985,U514,R290,U617,L697,U812,L633,U936,R214,D447,R509,D585,R787,D500,R305,D598,R866,U781,L771,D350,R558,U669,R284,D686,L231,U574,L539,D337,L135,D751,R315,D344,L694,D947,R118,U377,R50,U181,L96,U904,L776,D268,L283,U233,L757,U536,L161,D881,R724,D572,R322"""

In [101]:
import math

def toInstruction(text):
    return (text[0], int(text[1:]))

def toWire(text):
    return map(toInstruction, text.split(","))

def manhattan_distance(point, origin=(0,0)):
    (x, y) = point
    (x0, y0) = origin
    return abs(x-x0)+abs(y-y0)

class Day3:
    def solveA(self, text):
        wireA = toWire(text.split('\n')[0])
        wireB = toWire(text.split('\n')[1])
        grid = {}
        curPoint = (0,0) # lower-left
        for (d, dist) in wireA:
            (curX, curY) = curPoint
            (nextX, nextY) = (curX, curY)
            if d == 'U':
                nextY = curY + dist
                for y in range(curY, nextY+1):
                    grid[(curX,y)] = True
            elif d == 'D':
                nextY = curY - dist
                for y in range(curY, nextY-1, -1):
                    grid[(curX,y)] = True
            elif d == 'R':
                nextX = curX + dist
                for x in range(curX, nextX+1):
                    grid[(x,nextY)] = True
            elif d == 'L':
                nextX = curX - dist
                for x in range(curX, nextX-1, -1):
                    grid[(x,nextY)] = True
            # print("after {}, curPoint {} -> nextPoint {}".format((d,dist),curPoint,(nextX,nextY), grid))
            curPoint = (nextX, nextY)

        intersections = set()
        curPoint = (0,0) # lower-left
        for (d, dist) in wireB:
            (curX, curY) = curPoint
            (nextX, nextY) = (curX, curY)
            if d == 'U':
                nextY = curY + dist
                for y in range(curY, nextY+1):
                    if (curX, y) in grid:
                        intersections.add((curX,y))
            elif d == 'D':
                nextY = curY - dist
                for y in range(curY, nextY-1, -1):
                    if (curX,y) in grid:
                        intersections.add((curX,y))
            elif d == 'R':
                nextX = curX + dist
                for x in range(curX, nextX+1):
                    if (x,nextY) in grid:
                        intersections.add((x,nextY))
            elif d == 'L':
                nextX = curX - dist
                for x in range(curX, nextX-1, -1):
                    if (x,nextY) in grid:
                        intersections.add((x,nextY))
            curPoint = (nextX, nextY)
        intersections.remove((0,0))
        m_dists = map(manhattan_distance, intersections)
        # print(intersections)
        return min(m_dists)

    def solveB(self, text):
        wireA = toWire(text.split('\n')[0])
        wireB = toWire(text.split('\n')[1])
        grid = {}
        curPoint = (0,0) # lower-left
        wireADist = 0
        for (d, dist) in wireA:
            (curX, curY) = curPoint
            (nextX, nextY) = (curX, curY)
            if d == 'U':
                nextY = curY + dist
                for y in range(curY+1, nextY+1):
                    wireADist += 1
                    grid[(curX,y)] = min(wireADist, grid.get((curX,y), math.inf))
            elif d == 'D':
                nextY = curY - dist
                for y in range(curY-1, nextY-1, -1):
                    wireADist += 1
                    grid[(curX,y)] = min(wireADist, grid.get((curX,y), math.inf))
            elif d == 'R':
                nextX = curX + dist
                for x in range(curX+1, nextX+1):
                    wireADist += 1
                    grid[(x,curY)] = min(wireADist, grid.get((x,curY), math.inf))
            elif d == 'L':
                nextX = curX - dist
                for x in range(curX-1, nextX-1, -1):
                    wireADist += 1
                    grid[(x,curY)] = min(wireADist, grid.get((x,curY), math.inf))
            curPoint = (nextX, nextY)
#         print(grid)
        intersections = {}
        curPoint = (0,0) # lower-left
        wireBDist = 0
        for (d, dist) in wireB:
            (curX, curY) = curPoint
            (nextX, nextY) = (curX, curY)
            if d == 'U':
                nextY = curY + dist
                for y in range(curY+1, nextY+1):
                    wireBDist += 1
                    if (curX, y) in grid:
                        if (curX, y) not in intersections:
                            intersections[(curX, y)] = []
                        intersections[(curX,y)].append( (grid[(curX, y)], wireBDist) )
            elif d == 'D':
                nextY = curY - dist
                for y in range(curY-1, nextY-1, -1):
                    wireBDist += 1
                    if (curX,y) in grid:
                        if (curX, y) not in intersections:
                            intersections[(curX, y)] = []
                        intersections[(curX,y)].append( (grid[(curX, y)], wireBDist) )
            elif d == 'R':
                nextX = curX + dist
                for x in range(curX+1, nextX+1):
                    wireBDist += 1
                    if (x,curY) in grid:
                        if (x,curY) not in intersections:
                            intersections[(x,curY)] = []
                        intersections[(x,curY)].append( (grid[x, curY], wireBDist) )
            elif d == 'L':
                nextX = curX - dist
                for x in range(curX-1, nextX-1, -1):
                    wireBDist += 1
                    if (x,curY) in grid:
                        if (x,curY) not in intersections:
                            intersections[(x,curY)] = []
                        intersections[(x,curY)].append( (grid[x, curY], wireBDist) )
            curPoint = (nextX, nextY)
        minDist = math.inf
        for point in intersections:
            dists = intersections[point]
#             print("p {} dists {}".format(point, dists))
            for d in dists:
                (aDist, bDist) = d
                minDist = min( (aDist+bDist), minDist) 
#         print(intersections)
        return minDist
            
    
assert Day3().solveA("""R8,U5,L5,D3
U7,R6,D4,L4""") == 6
assert Day3().solveA("""R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83""") == 159
assert Day3().solveA("""R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7""") == 135

assert Day3().solveB("""R8,U5,L5,D3
U7,R6,D4,L4""") == 30
assert Day3().solveB("""R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83""") == 610
assert Day3().solveB("""R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7""") == 410

print(Day3().solveA(text), Day3().solveB(text))


248 28580


# [Day 4](https://adventofcode.com/2019/day/4)


In [102]:
text = "347312-805915"

In [103]:
class Day4:
    def isValidPass(self, number):
        digits = list(map(int, str(number)))
        if len(digits) is not 6:
            return False
        hasDuplicate = False
        for i in range(1,6):
            if digits[i-1] == digits[i]:
                hasDuplicate = True
            if digits[i] < digits[i-1]:
                return False
        return hasDuplicate
    
    def isValidStricterPass(self, number):
        digits = list(map(int, str(number)))
        if len(digits) is not 6:
            return False
        hasDuplicate = False
        for i in range(1,6):
            if digits[i-1] == digits[i]:
                if i > 1 and digits[i-2] == digits[i]:
                    pass
                elif i < 5 and digits[i+1] == digits[i]:
                    pass
                else:
                    hasDuplicate = True
            if digits[i] < digits[i-1]:
                return False
        return hasDuplicate

    def solveA(self, text):
        (lo, hi) = map(int, text.split("-"))
        count = 0
        for num in range(lo, hi):
            if self.isValidPass(num):
                count += 1
        return count
    
    def solveB(self, text):
        (lo, hi) = map(int, text.split("-"))
        count = 0
        for num in range(lo, hi):
            if self.isValidStricterPass(num):
                count += 1
        return count
        
    
assert Day4().isValidPass(111111)
assert not Day4().isValidPass(223450)
assert not Day4().isValidPass(123789)

assert Day4().solveA(text) == 594
assert Day4().solveB(text) == 364
print(Day4().solveA(text), Day4().solveB(text))

594 364


# [Day 5](https://adventofcode.com/2019/day/5)

In [11]:
text = "3,225,1,225,6,6,1100,1,238,225,104,0,1002,188,27,224,1001,224,-2241,224,4,224,102,8,223,223,1001,224,6,224,1,223,224,223,101,65,153,224,101,-108,224,224,4,224,1002,223,8,223,1001,224,1,224,1,224,223,223,1,158,191,224,101,-113,224,224,4,224,102,8,223,223,1001,224,7,224,1,223,224,223,1001,195,14,224,1001,224,-81,224,4,224,1002,223,8,223,101,3,224,224,1,224,223,223,1102,47,76,225,1102,35,69,224,101,-2415,224,224,4,224,102,8,223,223,101,2,224,224,1,224,223,223,1101,32,38,224,101,-70,224,224,4,224,102,8,223,223,101,3,224,224,1,224,223,223,1102,66,13,225,1102,43,84,225,1101,12,62,225,1102,30,35,225,2,149,101,224,101,-3102,224,224,4,224,102,8,223,223,101,4,224,224,1,223,224,223,1101,76,83,225,1102,51,51,225,1102,67,75,225,102,42,162,224,101,-1470,224,224,4,224,102,8,223,223,101,1,224,224,1,223,224,223,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,1108,226,677,224,1002,223,2,223,1005,224,329,101,1,223,223,108,226,226,224,1002,223,2,223,1005,224,344,1001,223,1,223,1107,677,226,224,1002,223,2,223,1006,224,359,101,1,223,223,1008,226,226,224,1002,223,2,223,1005,224,374,101,1,223,223,8,226,677,224,102,2,223,223,1006,224,389,101,1,223,223,7,226,677,224,1002,223,2,223,1005,224,404,1001,223,1,223,7,226,226,224,1002,223,2,223,1005,224,419,101,1,223,223,107,226,677,224,1002,223,2,223,1005,224,434,101,1,223,223,107,226,226,224,1002,223,2,223,1005,224,449,1001,223,1,223,1107,226,677,224,102,2,223,223,1006,224,464,1001,223,1,223,1007,677,226,224,1002,223,2,223,1006,224,479,1001,223,1,223,1107,677,677,224,1002,223,2,223,1005,224,494,101,1,223,223,1108,677,226,224,102,2,223,223,1006,224,509,101,1,223,223,7,677,226,224,1002,223,2,223,1005,224,524,1001,223,1,223,1008,677,226,224,102,2,223,223,1005,224,539,1001,223,1,223,1108,226,226,224,102,2,223,223,1005,224,554,101,1,223,223,107,677,677,224,102,2,223,223,1006,224,569,1001,223,1,223,1007,226,226,224,102,2,223,223,1006,224,584,101,1,223,223,8,677,677,224,102,2,223,223,1005,224,599,1001,223,1,223,108,677,677,224,1002,223,2,223,1005,224,614,101,1,223,223,108,226,677,224,102,2,223,223,1005,224,629,101,1,223,223,8,677,226,224,102,2,223,223,1006,224,644,1001,223,1,223,1007,677,677,224,1002,223,2,223,1006,224,659,1001,223,1,223,1008,677,677,224,1002,223,2,223,1005,224,674,101,1,223,223,4,223,99,226"

In [12]:
myprint = print

class IntCode:
    OPS = {
        1: 'add',
        2: 'mult',
        3: 'readinput',
        4: 'writeoutput',
        5: 'jumpiftrue',
        6: 'jumpiffalse',
        7: 'lessthan',
        8: 'equals',
        99: 'halt'
    }
    
    ADVANCE_COUNT = {
        1: 4,
        2: 4,
        3: 2,
        4: 2,
        5: 3,
        6: 3,
        7: 4,
        8: 4,
        99: 0
    }

    def __init__(self, mem=[], inputs=[]):
        self.mem = mem
        self.ip = 0
        self.done = False
        self.advanceIP = True
        self.inputs = inputs
        self.outputs = []
    
        
    def getOperand(self, ipOffset):
        if self.paramMode(ipOffset) == 'immediate':
            return self.mem[self.ip+ipOffset]
        else:
            return self.mem[self.getAddr(ipOffset)]
    
    def getAddr(self, ipOffset):
        return self.mem[self.ip + ipOffset]
    
    def jumpiftrue(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]

        if operands[0] != 0:
            self.ip = operands[1]
            self.advanceIP = False
            
    def jumpiffalse(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]

        if operands[0] == 0:
            self.ip = operands[1]
            self.advanceIP = False
            
    def lessthan(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getAddr(3)

        if operands[0] < operands[1]:
            self.mem[destAddr] = 1
        else:
            self.mem[destAddr] = 0
            
    def equals(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getAddr(3)

        if operands[0] == operands[1]:
            self.mem[destAddr] = 1
        else:
            self.mem[destAddr] = 0
        
    
    def add(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getAddr(3)
        self.mem[destAddr] = operands[0] + operands[1]

    def mult(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getAddr(3)
        self.mem[destAddr] = operands[0] * operands[1]
        
    def readinput(self):
        assert len(self.inputs) > 0, "Cannot read input, none left"
        _input = self.inputs.pop(0)
        destAddr = self.getAddr(1)
        self.mem[destAddr] = _input
        
    def writeoutput(self):
        value = self.getOperand(1)
        self.outputs.append(value)
        
    def unknown(self):
        print("Unknown opcode {}".format(self.opcode()))
        raise
        
    def halt(self):
        self.done = True
    
    def rawOpcode(self):
        return self.mem[self.ip]

    def paramMode(self, ipOffset):
        modes = list(reversed(list(map(int, str(self.rawOpcode())[:-2]))))
        if ipOffset - 1 >= len(modes):
            return 'position'
        elif modes[ipOffset - 1] == 0:
            return 'position'
        elif modes[ipOffset - 1] == 1:
            return 'immediate'
        assert False, "Unexpected paramMode {}".format(modes[ipOffset - 1])

    def opcode(self):
        return int(str(self.rawOpcode())[-2:])
    
    def op(self):
        op = getattr(self, IntCode.OPS.get(self.opcode(), 'unknown'))
        op()
        self.advance()
        
    def advance(self):
        if self.advanceIP:
            self.ip += IntCode.ADVANCE_COUNT[self.opcode()]
        self.advanceIP = True
    
    def getOperands(self):
        count = 0
        
    def run(self):
        while not self.done:
            self.op()
        return self.mem[0]

    
class Day5:
    def solve(self, text):
        mem = list(map(int, text.split(",")))
        inputs = [1]
        comp = IntCode(mem, inputs)
        comp.run()
        return comp.outputs[-1]
    def solveB(self, text):
        mem = list(map(int, text.split(",")))
        inputs = [5]
        comp = IntCode(mem, inputs)
        comp.run()
        return comp.outputs[-1]


assert IntCode(toIntList("1,9,10,3,2,3,11,0,99,30,40,50")).run() == 3500
assert IntCode(toIntList("1,0,0,0,99")).run() == 2
assert IntCode(toIntList("1,1,1,4,99,5,6,0,99")).run() == 30
IntCode(toIntList("1002,4,3,4,33")).run()

assert Day5().solve(text) == 13087969

# Examples for Part 2
eq8Prog = "3,9,8,9,10,9,4,9,99,-1,8"
c = IntCode(toIntList(eq8Prog), [7])
c.run()
assert c.outputs == [0]
c = IntCode(toIntList(eq8Prog), [8])
c.run()
assert c.outputs == [1]
c = IntCode(toIntList(eq8Prog), [9])
c.run()
assert c.outputs == [0]

lt8Prog = "3,9,7,9,10,9,4,9,99,-1,8"
c = IntCode(toIntList(lt8Prog), [7])
c.run()
assert c.outputs == [1]
c = IntCode(toIntList(lt8Prog), [8])
c.run()
assert c.outputs == [0]

eq8ProgImm = "3,3,1108,-1,8,3,4,3,99"
c = IntCode(toIntList(eq8ProgImm), [7])
c.run()
assert c.outputs == [0]
c = IntCode(toIntList(eq8ProgImm), [8])
c.run()
assert c.outputs == [1]

lt8ProgImm = "3,3,1107,-1,8,3,4,3,99"
c = IntCode(toIntList(lt8ProgImm), [7])
c.run()
assert c.outputs == [1]
c = IntCode(toIntList(lt8ProgImm), [8])
c.run()
assert c.outputs == [0]

print(
    Day5().solve(text),
    Day5().solveB(text),
)

13087969 14110739


# [Day 6](https://adventofcode.com/2019/day/6)

In [13]:
class Node:
    def __init__(self, label):
        self.label = label
        self.children = set()
        self.parent = None

    def parentCount(self):
        if self.parent is not None:
            return 1 + self.parent.parentCount()
        else:
            return 0

    def getParents(self):
        parents = []
        node = self.parent
        while node is not None:
            parents.append(node)
            node = node.parent
        return list(reversed(parents))
        
    
class Tree:
    def __init__(self):
        self.nodeLabels = {}
        
    def allNodes(self):
        return self.nodeLabels.values()

    def getNode(self, label):
        node = self.nodeLabels.get(label, Node(label))
        self.nodeLabels[label] = node
        return node
        
    def addChildOf(self, parentLabel, childLabel):
        child = self.getNode(childLabel)
        parent = self.getNode(parentLabel)
        assert child.parent is None, "Cannot add child if it already has parent"
        child.parent = parent
        parent.children.add(child)
        

class Day6:
    def parseTree(self, text):
        data = [item.split(')') for item in text.split('\n')]
        tree = Tree()
        for (parent, child) in data:
            tree.addChildOf(parent, child)
        return tree
    

    def solve(self, text):
        count = 0
        for node in self.parseTree(text).allNodes():
            count += node.parentCount()
        return count

    def solveB(self, text):
        tree = self.parseTree(text)
        me = tree.getNode('YOU')
        santa = tree.getNode('SAN')
        meParents = me.getParents()
        santaParents = santa.getParents()
        count = 0
        cur = meParents.pop()
        while cur is not None and cur not in santaParents:
            count += 1
            cur = meParents.pop()
        cur = santaParents.pop()
        while cur is not None and cur not in meParents:
            count +=1
            cur = santaParents.pop()
        # Subtract 1 because when we find the common branch, we will have popped
        # the common link
        return count - 1
        
assert Day6().solve("""COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L""") == 42

assert Day6().solveB("""COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN""") == 4

assert Day6().solve(readInput(6)) == 417916
assert Day6().solveB(readInput(6)) == 523

print(
    Day6().solve(readInput(6)), 
    Day6().solveB(readInput(6))
)

417916 523


# [Day 7](https://adventofcode.com/2019/day/7)

In [14]:
from itertools import permutations

class IntCode:
    OPS = {
        1: 'add',
        2: 'mult',
        3: 'readinput',
        4: 'writeoutput',
        5: 'jumpiftrue',
        6: 'jumpiffalse',
        7: 'lessthan',
        8: 'equals',
        9: 'relativebaseoffset',
        99: 'halt'
    }
    
    ADVANCE_COUNT = {
        1: 4,
        2: 4,
        3: 2,
        4: 2,
        5: 3,
        6: 3,
        7: 4,
        8: 4,
        9: 2,
        99: 0
    }

    def __init__(self, mem=[], inputs=[], readInput=None):
        self.counter = 0
        self.maxCount = None
        self.mem = {idx:val for idx,val in enumerate(mem)}
        self.ip = 0
        self.done = False
        self.advanceIP = True
        self.relativeBase = 0
        self.inputs = inputs
        self.readInput = readInput
        self.outputs = []
    
    def getFromMem(self, addr):
        assert addr >= 0, "Expected non-negative address, got {}".format(addr)
        return self.mem.get(addr, 0)

    def getOperand(self, ipOffset):
        if self.paramMode(ipOffset) == 'immediate':
            return self.getFromMem(self.ip+ipOffset)
        elif self.paramMode(ipOffset) == 'relative':
            return self.getFromMem(self.relativeBase + self.getFromMem(self.ip + ipOffset))
        else:
            return self.getFromMem(self.getFromMem(self.ip + ipOffset))
    
    def getDestAddr(self, ipOffset):
        if self.paramMode(ipOffset) == 'immediate':
            assert False, "Cannot have immediate-mode dest addr"
        elif self.paramMode(ipOffset) == 'relative':
            return self.relativeBase + self.getFromMem(self.ip + ipOffset)
        else:
            return self.getFromMem(self.ip + ipOffset)
        
    def relativebaseoffset(self):
        operand = self.getOperand(1)
        self.relativeBase += operand

    def jumpiftrue(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]

        if operands[0] != 0:
            self.ip = operands[1]
            self.advanceIP = False
            
    def jumpiffalse(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]

        if operands[0] == 0:
            self.ip = operands[1]
            self.advanceIP = False
            
    def lessthan(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getDestAddr(3)

        if operands[0] < operands[1]:
            self.mem[destAddr] = 1
        else:
            self.mem[destAddr] = 0
            
    def equals(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getDestAddr(3)

        if operands[0] == operands[1]:
            self.mem[destAddr] = 1
        else:
            self.mem[destAddr] = 0
    
    def add(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getDestAddr(3)
        self.mem[destAddr] = operands[0] + operands[1]

    def mult(self):
        operands = [
            self.getOperand(1),
            self.getOperand(2)
        ]
        destAddr = self.getDestAddr(3)
        self.mem[destAddr] = operands[0] * operands[1]
        
    def readinput(self):
        if self.readInput:
            _input = self.readInput()
        else:
            assert len(self.inputs) > 0, "Cannot read input, none left"
            _input = self.inputs.pop(0)
        destAddr = self.getDestAddr(1)
        self.mem[destAddr] = _input
        
    def writeoutput(self):
        value = self.getOperand(1)
        self.outputs.append(value)
        return value
        
    def unknown(self):
        print("Unknown opcode {}".format(self.opcode()))
        raise
        
    def halt(self):
        self.done = True
    
    def rawOpcode(self):
        return self.getFromMem(self.ip)

    def paramMode(self, ipOffset):
        modes = list(reversed(list(map(int, str(self.rawOpcode())[:-2]))))
        if ipOffset - 1 >= len(modes):
            return 'position'
        elif modes[ipOffset - 1] == 0:
            return 'position'
        elif modes[ipOffset - 1] == 1:
            return 'immediate'
        elif modes[ipOffset - 1] == 2:
            return 'relative'
        assert False, "Unexpected paramMode {}".format(modes[ipOffset - 1])

    def opcode(self):
        return int(str(self.rawOpcode())[-2:])
    
    def op(self):
        self.counter += 1
        if self.maxCount is not None and self.counter > self.maxCount:
            assert False, "Too many instructions {}".format(self.counter)
        op = getattr(self, IntCode.OPS.get(self.opcode(), 'unknown'))
        return op()
        
    def advance(self):
        if self.advanceIP:
            self.ip += IntCode.ADVANCE_COUNT[self.opcode()]
        self.advanceIP = True
    
    def runUntilOutput(self):
        while not self.done:
            output = self.op()
            self.advance()
            if output is not None:
                return output
        
    def run(self):
        while not self.done:
            self.op()
            self.advance()



In [15]:
class Day7:
    def solve(self, text):
        maxOutput = 0
        maxSettingSeq = None

        for settingSeq in permutations(range(5),5):
            settingSeq = list(settingSeq)
            mem = toIntList(text, sep=",")
            _input = 0
            for idx, amplifier in enumerate(['A','B','C','D','E']):
                inputs = [settingSeq[idx], _input]
                comp = IntCode(mem, inputs)
                comp.run()
                _input = comp.outputs[-1]
            if _input > maxOutput:
                maxOutput = _input
                maxSettingSeq = settingSeq
        return (maxOutput, maxSettingSeq)

    def solveB(self, text):
        maxOutput = 0
        maxSettingSeq = None

        for settingSeq in permutations(range(5, 10)):
            settingSeq = list(settingSeq)
            mem = toIntList(text, sep=",")
            
            comps = [IntCode(list(mem), inputs=[]) for _ in range(5)]
            for idx,phase in enumerate(settingSeq):
                comps[idx].inputs.append(phase)
            comps[0].inputs.append(0)

            done = False
            runCount = 0
            finalOutput = 0
            while not done:
                runCount += 1
                for idx,comp in enumerate(comps):
                    _inputs = list(comp.inputs)
                    output = comp.runUntilOutput()
                    if output is None:
                        done = True
                    else:
                        comps[ (idx + 1) % 5].inputs.append(output)
            finalOutput = comps[-1].outputs[-1]
            if finalOutput > maxOutput:
                maxOutput = finalOutput
                maxSettingSeq = settingSeq
        return (maxOutput, maxSettingSeq)
    
        

assert Day7().solve(
    "3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0"
) == (43210, [4, 3, 2, 1, 0])
assert Day7().solve(
    "3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0"
) == (54321, [0,1,2,3,4])
assert Day7().solve(
    "3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0"
) == (65210, [1,0,4,3,2])


assert Day7().solveB(
    "3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5"
) == (139629729, [9, 8, 7, 6, 5])
assert Day7().solveB(
    "3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10"
) == (18216, [9,7,8,5,6])

assert Day7().solve(readInput(7)) == (13848, [2, 0, 3, 1, 4])
assert Day7().solveB(readInput(7)) == (12932154, [6, 8, 7, 5, 9])

print(
    Day7().solve(readInput(7)),
    Day7().solveB(readInput(7)),
)


(13848, [2, 0, 3, 1, 4]) (12932154, [6, 8, 7, 5, 9])


# [Day 8](https://adventofcode.com/2019/day/8)

In [16]:
from collections import Counter
from math import inf

class Day8:    
    def solve(self, text, width=25, height=6):
        idx = 0
        data = toIntList(text, sep=None)
        
        layers = []
        layer = []
        while idx < len(data):
            for col in range(width):
                for row in range(height):
                    layer.append(data[idx])
                    idx += 1
            layers.append(layer)
            layer = []
        
        minZeros = inf
        minIndex = None
        for idx, layer in enumerate(layers):
            counter = Counter(layer)
            if counter[0] < minZeros:
                minZeros = counter[0]
                minIndex = idx
        counter = Counter(layers[minIndex])
        return counter[1] * counter[2]

    def solveB(self, text, width=25, height=6):
        idx = 0
        data = toIntList(text, sep=None)
        
        layers = []
        layer = []
        while idx < len(data):
            for col in range(width):
                for row in range(height):
                    layer.append(data[idx])
                    idx += 1
            layers.append(layer)
            layer = []

        
        for layer in layers:
            assert len(layer) == width * height

        output = []
        transparent = 2

        for row in range(height):
            for col in range(width):
                idx = row*width + col
                color = 2
                for layer in layers:
                    if layer[idx] != transparent:
                        color = layer[idx]
                        break
                output.append(color)

        outputStr = ""
        blackSquare = "▮"
        for row in range(height):
            for col in range(width):
                idx = row*width + col
                if output[idx] == 1:
                    outputStr += blackSquare
                else:
                    outputStr += " "
            outputStr += "\n"
                
        return outputStr

        
assert Day8().solve(readInput(8)) == 2460

print(Day8().solve(readInput(8)))
print(Day8().solveB(readInput(8)))

2460
▮    ▮▮▮  ▮▮▮▮ ▮  ▮ ▮  ▮ 
▮    ▮  ▮ ▮    ▮ ▮  ▮  ▮ 
▮    ▮  ▮ ▮▮▮  ▮▮   ▮  ▮ 
▮    ▮▮▮  ▮    ▮ ▮  ▮  ▮ 
▮    ▮ ▮  ▮    ▮ ▮  ▮  ▮ 
▮▮▮▮ ▮  ▮ ▮    ▮  ▮  ▮▮  



# [Day 9](https://adventofcode.com/2019/day/9)

In [17]:
def testProg(name, progString, inputs, expectedOutputs):
    prog = toIntList(progString, sep=",")
    comp = IntCode(prog, inputs)
    comp.run()
    assert comp.outputs == expectedOutputs
    print("prog {} OK".format(name))


quineProg = "109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99"
testProg("Quine", quineProg, [], toIntList(quineProg, sep=","))

output16DigitProg = "1102,34915192,34915192,7,4,7,99,0"
testProg("16-Digit", output16DigitProg, [], [1219070632396864])

testProg("Big Number", "104,1125899906842624,99", [], [1125899906842624])

# Uses Day 7 IntCode comp

class Day9:
    def solve(self, text):
        prog = toIntList(text, sep=",")
        comp = IntCode(prog, [1])
        comp.run()
        return comp.outputs[0]
    def solveB(self, text):
        prog = toIntList(text, sep=",")
        comp = IntCode(prog, [2])
        comp.run()
        return comp.outputs[0]

assert Day9().solve(readInput(9)) == 2453265701
assert Day9().solveB(readInput(9)) == 80805
print(
    Day9().solve(readInput(9)),
    Day9().solveB(readInput(9))
)

prog Quine OK
prog 16-Digit OK
prog Big Number OK
2453265701 80805


# [Day 10](https://adventofcode.com/2019/day/10)

In [49]:
import math

def rad2deg(rad):
    return (180/math.pi)*rad

def getAngle(p0, p1):
    rise = p0[1] - p1[1]
    run = p1[0] - p0[0]
    rad = math.atan2(run, rise)
    deg = rad2deg(rad)
    if deg < 0:
        deg += 360
    return deg
    

class Day10:
    def parsePoints(self, text):
        points = set()
        for y,row in enumerate(text.split("\n")):
            for x,dot in enumerate(row.strip()):
                if dot == '#':
                    points.add((x,y))
        return points

    def solve(self, text):
        points = self.parsePoints(text)
        height = len(text.split('\n'))
        width = len(text.split('\n')[0])
        dimensions = (width, height)
        maxDim = max(width, height)
        maxCount = 0
        maxPoint = None
        for p0 in points:
            angles = set()
            count = 0
            for dist in range(1, maxDim):
                for p1 in gridPointsAtDistance(p0, dimensions, dist):
                    if p1 in points:
                        angle = getAngle(p0, p1)
                        if angle not in angles:
                            count += 1
                        angles.add(angle)
            if count > maxCount:
                maxCount = count
                maxPoint = p0
        return (maxPoint, maxCount)
    def solveB(self, text):
        centralPoint = self.solve(text)[0]
        points = self.parsePoints(text)
        height = len(text.split('\n'))
        width = len(text.split('\n')[0])
        dimensions = (width, height)
        maxDim = max(width, height)
        maxCount = 0
        maxPoint = None
        angleMap = {}

        p0 = centralPoint
        angles = set()
        count = 0
        point2AngleMap = {}
        for dist in range(1, maxDim):
            for p1 in gridPointsAtDistance(p0, dimensions, dist):
                if p1 in points:
                    angle = getAngle(p0, p1)
                    point2AngleMap[p1] = angle
                    if angle not in angleMap:
                        angleMap[angle] = []
                    angleMap[angle].append(p1)
        hasPointsRemaining = True
        destroyedPoints = []
        while hasPointsRemaining:
            hasPointsRemaining = False
            for angle in sorted(angleMap.keys()):
                if len(angleMap[angle]) > 0:
                    p1 = angleMap[angle].pop(0)
                    destroyedPoints.append(p1)
                if len(angleMap[angle]) > 0:
                    hasPointsRemaining = True
        return destroyedPoints

    
assert Day10().solve(
    """.#..#
    .....
    #####
    ....#
    ...##"""
) == ((3,4), 8)

assert Day10().solve("""......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####""") == ((5,8), 33)

assert Day10().solve("""#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###.""") == ((1,2), 35)

assert Day10().solve(""".#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#..""") == ((6,3), 41)

assert Day10().solve(""".#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##""") == ((11,13), 210)


res = Day10().solveB(""".#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##""")

assert res[0] == (11,12)
assert res[1] == (12,1)
assert res[2] == (12,2)
assert res[9] == (12,8)
assert res[19] == (16,0)
assert res[49] == (16,9)
assert res[99] == (10,16)
assert res[199] == (8,2)
assert res[200] == (10,9)
assert res[298] == (11,1)
assert len(res) == 299

assert Day10().solve(readInput(10)) == ((26,29), 303)
assert Day10().solveB(readInput(10))[199] == (4,8)

print(
    Day10().solve(readInput(10)),
    Day10().solveB(readInput(10))[199]
)

((26, 29), 303) (4, 8)


# [Day 11](https://adventofcode.com/2019/day/11)

In [19]:
from collections import defaultdict

BLACK = 0
WHITE = 1
L90 = 0
R90 = 1
D_N = 0
D_E = 1
D_S = 2
D_W = 3
DIRS = [D_N, D_E, D_S, D_W]

def adjustCoords(coords, facing, newDir):
    assert newDir in [L90,R90]
    assert facing in DIRS
    (x,y) = coords

    if newDir == L90:
        facing = DIRS[ DIRS[facing] - 1 ]
    else:
        facing = DIRS[ (DIRS[facing] + 1) % len(DIRS) ]
    if facing == D_N:
        y += 1
    elif facing == D_E:
        x += 1
    elif facing == D_S:
        y -= 1
    elif facing == D_W:
        x -= 1
    else:
        assert False
    
    return ( (x,y), facing )

assert adjustCoords((0,0), D_N, R90) == ((1,0), D_E)
assert adjustCoords((0,0), D_E, R90) == ((0,-1), D_S)
assert adjustCoords((-2,3), D_W, L90) == ((-2,2), D_S)


class Day11:
    def solve(self, text):
        facing = D_N
        grid = defaultdict(lambda: BLACK)
        coords = (0,0)
        seen = set()
        
        prog = toIntList(text, sep=",")
        comp = IntCode(prog, [])    
        
        while not comp.done:
            comp.inputs.append(grid[coords])
            newColor = comp.runUntilOutput()
            if newColor is None:
                break
            assert newColor in [BLACK,WHITE]
            grid[coords] = newColor
            seen.add(coords)

            newDir = comp.runUntilOutput()
            (coords, facing) = adjustCoords(coords, facing, newDir)
        return len(seen)
    
    def solveB(self, text):
        facing = D_N
        grid = defaultdict(lambda: BLACK)
        coords = (0,0)
        grid[coords] = WHITE
        
        prog = toIntList(text, sep=",")
        comp = IntCode(prog, [])    
        
        while not comp.done:
            comp.inputs.append(grid[coords])
            newColor = comp.runUntilOutput()
            if newColor is None:
                break
            assert newColor in [BLACK,WHITE]
            grid[coords] = newColor

            newDir = comp.runUntilOutput()
            (coords, facing) = adjustCoords(coords, facing, newDir)

        minX = min([x for (x,y) in grid.keys()])
        maxX = max([x for (x,y) in grid.keys()])
        minY = min([y for (x,y) in grid.keys()])
        maxY = max([y for (x,y) in grid.keys()])
        
        # Invert these for contrast
        W_SQ = "⬛"
        B_SQ = "⬜"
        lines = []

        # North was -Y, South was +Y, so invert these when
        # building the string
        for y in range(maxY, minY-1, -1):
            line = ""
            for x in range(minX, maxX+1):
                color = grid[(x,y)]
                if color == BLACK:
                    line += B_SQ
                else:
                    line += W_SQ
            lines.append(line)
            
            
        return "\n".join(lines)

assert Day11().solve(readInput(11)) == 1932
print(
    Day11().solve(readInput(11))
)
print(Day11().solveB(readInput(11)))

1932
⬜⬛⬛⬛⬛⬜⬜⬛⬛⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬜⬛⬛⬜⬜⬜⬜⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬛⬜⬜⬛⬜⬜⬛⬜⬜⬜⬜⬛⬜⬛⬜⬜⬜⬜⬛⬜⬜⬛⬜⬜⬜
⬜⬛⬛⬛⬜⬜⬛⬜⬜⬜⬜⬛⬛⬛⬛⬜⬛⬛⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬛⬜⬛⬛⬛⬜⬜⬛⬜⬜⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬛⬛⬜⬛⬜⬜⬛⬜⬛⬜⬛⬜⬜⬛⬜⬛⬛⬜⬜⬜⬜⬛⬜⬛⬜⬜⬜⬜⬛⬛⬛⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬛⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬜⬜⬛⬜⬛⬜⬜⬜⬜
⬜⬛⬛⬛⬛⬜⬜⬛⬛⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬜⬛⬛⬛⬜⬜⬛⬛⬜⬜⬛⬛⬛⬛⬜⬛⬜⬜⬛⬜⬜⬜


# [Day 12](https://adventofcode.com/2019/day/12)

In [20]:
from itertools import combinations

class System:
    def __init__(self, moons):
        self.moons = moons

    def tick(self):
        # gravity
        for (m1, m2) in combinations(self.moons, 2):
            for axis in range(3):
                m1p = m1.p[axis]
                m2p = m2.p[axis]
                if m1p == m2p:
                    pass
                else:
                    if m1p > m2p:
                        m2.v[axis] += 1
                        m1.v[axis] -= 1
                    else:
                        m2.v[axis] -= 1
                        m1.v[axis] += 1
        # velocity
        for m in self.moons:
            m.move()
            
    def tickAxis(self, axis):
        assert 0 <= axis < 3
        # gravity
        for (m1, m2) in combinations(self.moons, 2):
            m1p = m1.p[axis]
            m2p = m2.p[axis]
            if m1p == m2p:
                pass
            else:
                if m1p > m2p:
                    m2.v[axis] += 1
                    m1.v[axis] -= 1
                else:
                    m2.v[axis] -= 1
                    m1.v[axis] += 1
        # velocity
        for m in self.moons:
            m.moveAxis(axis)

    def energy(self):
        return sum([m.totalE() for m in self.moons])
    
    def stateAxis(self, axis):
        state = []
        for m in self.moons:
            state.append(m.p[axis])
            state.append(m.v[axis])
        return state
    
class Moon:
    def __init__(self, position):
        self.p = position
        self.v = [0,0,0]
        
    def move(self):
        for axis in range(3):
            self.p[axis] += self.v[axis]
            
    def moveAxis(self, axis):
        assert 0 <= axis < 3
        self.p[axis] += self.v[axis]

    def totalE(self):
        return self.kineticE() * self.potentialE()
    
    def potentialE(self):
        e = 0
        for axis in range(3):
            e += abs(self.p[axis])
        return e

    def kineticE(self):
        e = 0
        for axis in range(3):
            e += abs(self.v[axis])
        return e
    
    def __repr__(self):
        [px,py,pz] = self.p
        [vx,vy,vz] = self.v
        return "Moon <pos=<x={}, y={}, z={}>> <velocity=<x={}, y={}, z={}>>".format(px,py,pz,vx,vy,vz)

class Day12:
    def solve(self, moons):
        s = System(moons)
        for _ in range(1000):
            s.tick()
        return s.energy()

    def solveB(self, moons):
        s = System(moons)
        initialStates = {}
        periods = {}
        for axis in range(3):
            initialStates[axis] = s.stateAxis(axis)
            count = 0
            while True:
                count += 1
                s.tickAxis(axis)
                if s.stateAxis(axis) == initialStates[axis]:
                    periods[axis] = count
                    break
        return lcmm(*periods.values())
            

# Raw Input
"""
<x=8, y=0, z=8>
<x=0, y=-5, z=-10>
<x=16, y=10, z=-5>
<x=19, y=-10, z=-7>
"""
day12Moons = [
    Moon([8,0,8]),
    Moon([0,-5,-10]),
    Moon([16,10,-5]),
    Moon([19,-10,-7])
]


example1 = [
    Moon([-1,0,2]),
    Moon([2,-10,-7]),
    Moon([4,-8,8]),
    Moon([3,5,-1])
]

example2 = [
    Moon([-8, -10, 0]),
    Moon([5, 5, 10]),
    Moon([2, -7, 3]),
    Moon([9, -8, -3])
]

assert Day12().solveB(example1) == 2772
assert Day12().solveB(example2) == 4686774924
assert Day12().solveB(day12Moons) == 392733896255168

print(
    Day12().solve(day12Moons),
    Day12().solveB(day12Moons)
)


12490 392733896255168


# [Day 13](https://adventofcode.com/2019/day/13)

In [21]:
T_EMPTY = 0
T_WALL = 1
T_BLOCK = 2
T_PADDLE = 3
T_BALL = 4
TILES = [T_EMPTY, T_WALL, T_BLOCK, T_PADDLE, T_BALL]

X_SCORE = -1
Y_SCORE = 0

J_NEUTRAL = 0
J_LEFT = -1
J_RIGHT = 1

class Day13:
    def solve(self, text):
        prog = toIntList(text, sep=",")
        comp = IntCode(prog, [])
        grid = {}
        count = 0
        while not comp.done:
            x = comp.runUntilOutput()
            if x is None:
                break
            y = comp.runUntilOutput()
            assert y is not None
            tile = comp.runUntilOutput()
            assert tile in TILES
            if tile == T_BLOCK:
                count += 1
            grid[(x,y)] = tile
        return len([t for t in grid.values() if t == T_BLOCK])
    
    def solveB(self, text):
        prog = toIntList(text, sep=",")
        prog[0] = 2
        ball_pos = None
        paddle_pos = None
        def readInput():
            _input = None
            if ball_pos is None or paddle_pos is None:
                _input = J_NEUTRAL
            else:
                if ball_pos[0] == paddle_pos[0]:
                    _input = J_NEUTRAL
                elif ball_pos[0] < paddle_pos[0]:
                    _input = J_LEFT
                else:
                    _input = J_RIGHT
            return _input
        
        comp = IntCode(prog, [], readInput=readInput)
        score = 0
        while not comp.done:
            x = comp.runUntilOutput()
            if x is None:
                break
            y = comp.runUntilOutput()
            assert y is not None
            if x == X_SCORE:
                assert y == Y_SCORE
                score = comp.runUntilOutput()
            else:
                tile = comp.runUntilOutput()
                assert tile in TILES
                if tile == T_BALL:
                    ball_pos = (x,y)
                if tile == T_PADDLE:
                    paddle_pos = (x,y)
        return score
    
assert Day13().solve(readInput(13)) == 420
assert Day13().solveB(readInput(13)) == 21651

print(
    Day13().solve(readInput(13)),
    Day13().solveB(readInput(13))
)

420 21651


# [Day 14](https://adventofcode.com/2019/day/14)

In [22]:
class Chemistry:
    def parseReaction(self, text):
        inputs = []
        output = None
        (lhs, rhs) = text.split(' => ')
        for pair in lhs.split(', '):
            (num, name) = pair.split(' ')
            num = int(num)
            inputs.append((num, name))
        (num, name) = rhs.split(' ')
        output = (int(num), name)
        return (inputs, output)
        
    def parse(self, text):
        reactions = [self.parseReaction(line.strip()) for line in text.strip().split('\n')]
        outputs = {}
        for r in reactions:
            (num, name) = r[1]
            assert name not in outputs
            outputs[name] = (num, r[0])
            
        return outputs
        

assert Chemistry().parseReaction(
    "9 ORE => 2 A"
) == ([ (9, 'ORE') ] , (2, 'A'))
assert Chemistry().parseReaction(
    "3 A, 4 B => 1 AB"
) == ([ (3, 'A'), (4, 'B') ] , (1, 'AB'))
assert Chemistry().parseReaction(
    "2 AB, 3 BC, 4 CA => 1 FUEL"
) == ([(2,'AB'),(3,'BC'),(4,'CA')], (1, 'FUEL'))


class Day14:
    def solve(self, text, neededFuel=1):
        self.outputs = Chemistry().parse(text)
        self.have = defaultdict(lambda: 0)
        self.need = defaultdict(lambda: 0)
        self.ore = 0
        self.need['FUEL'] = neededFuel
        while sum(self.need.values()) != 0:
            for (name,num) in list(self.need.items()):
                self.produce(name,num)
        return self.ore

    def produce(self,name,num):
        if name == 'ORE':
            self.ore += num
            self.need['ORE'] -= num
        else:
            self.need[name] -= num
            assert self.need[name] >= 0
            # First use what is available
            used = min(num, self.have[name])
            self.have[name] -= used
            num -= used
            assert num >= 0
            if used > 0:
                # print("used {} of {}, still need {}".format(used, name, num))
                pass


            # If we still need some, perform the rxn
            if num > 0:
                # find the rxn that produces name
                (rxnOutNum, rxn) = self.outputs[name]
                multiplier = math.ceil(num / rxnOutNum)
                created = rxnOutNum * multiplier
                assert self.have[name] == 0
                self.have[name] = created
                assert self.have[name] >= num
                self.have[name] -= num
                for (rxInNum,rxInName) in rxn:
                    amtNeeded = multiplier * rxInNum
                    self.need[rxInName] += amtNeeded

    def solveB(self, text):
        trillion = 1000000000000
        neededFuel = 10000
        neededOre = 0
        
        # establish lower and upper bounds
        while neededOre < trillion:
            neededFuel = neededFuel * 2
            neededOre = Day14().solve(text, neededFuel)
        lower = neededFuel // 2
        while neededOre > trillion:
            neededFuel = neededFuel // 2
            neededOre = Day14().solve(text, neededFuel)
        upper = neededFuel * 2
        
        # binary search
        while True:
            window = upper - lower
            midpoint = lower + (window // 2)
            neededOre = Day14().solve(text, midpoint)
            if neededOre > trillion:
                upper = midpoint
            elif neededOre == trillion:
                return midpoint
            else:
                lower = midpoint
                
            if window <= 1:
                return midpoint
            

example1 = """
10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
"""
assert Day14().solve(example1) == 31

example2 = """9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL"""
assert Day14().solve(example2) == 165

example3 = """157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT"""
assert Day14().solve(example3) == 13312

example4 = """2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF"""
assert Day14().solve(example4) == 180697

example5 = """171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX"""
assert Day14().solve(example5) == 2210736


assert Day14().solve(readInput(14)) == 365768

assert Day14().solveB(example3) == 82892753
assert Day14().solveB(example4) == 5586022
assert Day14().solveB(example5) == 460664
assert Day14().solveB(readInput(14)) == 3756877

print(
    Day14().solve(readInput(14)),
    Day14().solveB(readInput(14))
)


365768 3756877


# [Day 15](https://adventofcode.com/2019/day/15)

In [46]:
(N, S, W, E) = (1,2,3,4)
DIRS=[N,S,W,E]
(WALL,MOVE,OXYGEN) = (0,1,2)
RESPONSES = [WALL, MOVE, OXYGEN]
UNKNOWN = 4
POS_TYPES = [WALL, MOVE, OXYGEN, UNKNOWN]

class Day15:
    def solve(self, text):
        self.grid = defaultdict(lambda: UNKNOWN)
        prog = toIntList(text, sep=",")
        self.comp = IntCode(prog, inputs=[])
        self.seen = set()
        return self.dfs((0,0),path=[],dist=0)

    def solveB(self, text):
        self.grid = defaultdict(lambda: UNKNOWN)
        prog = toIntList(text, sep=",")
        self.comp = IntCode(prog, inputs=[])
        self.dfs2((0,0),path=[],dist=0)
        
        oxy_pos = list(self.grid.keys())[list(self.grid.values()).index(OXYGEN)]
        SEEN = -1
        self.grid[oxy_pos] = SEEN
        seen = set([oxy_pos])
        last_seen = set([oxy_pos])
        done = MOVE not in self.grid.values()
        minutes = 0
        while not done:
            next_seen = set()
            for pos in last_seen:
                for n in gridNeighbors(self.grid, pos):
                    if self.grid[n] == MOVE and n not in seen:
                        next_seen.add(n)
            for pos in next_seen:
                self.grid[pos] = SEEN
            minutes += 1
            seen = seen.union(last_seen)
            last_seen = next_seen
            done = MOVE not in self.grid.values()
        return minutes

    
    # DFS that stops when it finds OXYGEN
    def dfs(self, pos, path=[], dist=0):
        neighbors = [n for n in gridNeighbors(self.grid, pos) if self.grid[n] == UNKNOWN]
        for n in neighbors:
            self.comp.inputs.append(self.dirTo(pos,n))
            resp = self.comp.runUntilOutput()
            assert resp in RESPONSES
            self.grid[n] = resp
            if resp == OXYGEN:
                return dist + 1
            elif resp == MOVE:
                return self.dfs(n, path+[pos], dist+1)
        assert len(path) > 0
        assert dist > 0
        prevPos = path[-1]
        self.comp.inputs.append(self.dirTo(pos, prevPos))
        resp = self.comp.runUntilOutput()
        assert resp == MOVE
        return self.dfs(prevPos, path[0:-1], dist - 1)           

    # DFS that maps the entire area
    def dfs2(self, pos, path=[], dist=0):
        neighbors = [n for n in gridNeighbors(self.grid, pos) if self.grid[n] == UNKNOWN]
        for n in neighbors:
            self.comp.inputs.append(self.dirTo(pos,n))
            resp = self.comp.runUntilOutput()
            assert resp in RESPONSES
            self.grid[n] = resp
            if resp in [OXYGEN, MOVE]:
                return self.dfs2(n, path+[pos], dist+1)
        if len(path) == 0:
            # Nowhere to backtrack to
            return
        else:
            # Backtrack
            prevPos = path[-1]
            self.comp.inputs.append(self.dirTo(pos, prevPos))
            resp = self.comp.runUntilOutput()
            assert resp == MOVE
            return self.dfs2(prevPos, path[0:-1], dist - 1)           

    
    def dirTo(self, fromPos, toPos):
        (fX,fY) = fromPos
        (tX,tY) = toPos
        dX = tX-fX
        dY = tY-fY
        assert abs(dX+dY) == 1, "dX {} + dY {} == 1".format(dX,dY)
        if dX == 1:
            return E
        elif dX == -1:
            return W
        elif dY == -1:
            return N
        elif dY == 1:
            return S
        else:
            assert False, "No dirTo from {} to {}".format(fromPos,toPos)
        
assert Day15().solve(readInput(15)) == 298
assert Day15().solveB(readInput(15)) == 346

print(
    Day15().solve(readInput(15)),
    Day15().solveB(readInput(15))
)

298 346


# [Day 16](https://adventofcode.com/2019/day/16)

In [24]:
import collections, itertools

class Day16:
    def solve(self, text):
        nums = list(map(int, text.strip()))
        nums = FFT(nums).nth(100)[:8]
        return ''.join(map(str, nums))

def fftSeq(n):
    def _(n):
        baseIdx = 0
        base = [0, 1, 0, -1]
        idx = 0
        while True:
            yield base[baseIdx]
            idx += 1
            if idx >= n:
                idx = 0
                baseIdx += 1
            if baseIdx >= len(base):
                baseIdx = 0
    gen = _(n)
    next(gen)
    return gen

def onesDigit(num):
    num = abs(num)
    return int(str(num)[-1])
    
assert onesDigit(5) == 5
assert onesDigit(-5) == 5
assert onesDigit(25) == 5
assert onesDigit(-25) == 5

class FFT:
    def __init__(self, nums):
        self.nums = nums
        
    def tick(self):
        next_nums = []
        for idx in range(len(self.nums)):
            idx += 1
            gen = fftSeq(idx)
            num = onesDigit(sum([x*y for (x,y) in zip(self.nums, gen)]))
            next_nums.append(num)
        self.nums = next_nums
        return self.nums
        
    def nth(self, n):
        for i in range(n):
            self.tick()
        return self.nums

        
assert FFT([1,2,3,4,5,6,7,8]).nth(1) == [4,8,2,2,6,1,5,8]
assert FFT([1,2,3,4,5,6,7,8]).nth(2) == [3,4,0,4,0,4,3,8]
assert FFT([1,2,3,4,5,6,7,8]).nth(3) == [0,3,4,1,5,5,1,8]
assert FFT([1,2,3,4,5,6,7,8]).nth(4) == [0, 1, 0, 2, 9, 4, 9, 8]

assert Day16().solve("80871224585914546619083218645595") == "24176176"
assert Day16().solve("19617804207202209144916044189917") == "73745418"
assert Day16().solve("69317163492948606335995924319873") == "52432133"
assert Day16().solve(readInput(16)) == '34841690'

print(
    Day16().solve(readInput(16))
)

34841690


# [Day 17](https://adventofcode.com/2019/day/17)

In [25]:
(SCAFFOLD, SPACE, NEWLINE, LOST, D_N, D_E, D_S, D_W) = [
    ord(x) for x in ["#",".","\n","X","^",">","v","<"]
]

RESPONSES = [SCAFFOLD, SPACE, NEWLINE, LOST, D_N, D_E, D_S, D_W]

def getNeighbors(pos, grid):
    (x,y) = pos
    cardinal_dirs = {
        N: (-1,0),
        E: (0,1),
        S: (1,0),
        W: (0,-1)
    }
    
    neighbor_positions = [
        (x+dX,y+dY) for (dX,dY) in cardinal_dirs.values()
    ]
    return [n for n in neighbor_positions if grid[n] is not None]
        

class Day17:
    def solve(self, text):
        prog = toIntList(text, sep=",")
        comp = IntCode(prog, inputs=[])
        grid = defaultdict(lambda:None)
        x = 0
        y = 0
        while not comp.done:
            resp = comp.runUntilOutput()
            if resp == None:
                break
            assert resp in RESPONSES, "Expect {} in {}".format(resp,RESPONSES)
            if resp == NEWLINE:
                y += 1
                x = 0
            else:
                grid[(x,y)] = resp
                x += 1
        intersections = set()
        for (pos,value) in list(grid.items()):
            if value == SCAFFOLD:
                if all([grid[n] == SCAFFOLD for n in getNeighbors(pos, grid)]):
                    intersections.add(pos)
        return sum([x*y for (x,y) in intersections])
            
        
assert Day17().solve(readInput(17)) == 7404

print(
    Day17().solve(readInput(17))
)

7404


# [Day 18](https://adventofcode.com/2019/day/18)

In [ ]:
WALL = '#'
SPACE = '.'
ENTRANCE = '@'

def lockForKey(key):
    assert key.islower()
    assert len(key) == 1
    return key.upper()

def isKey(char):
    assert len(char) == 1
    return char.islower()


def process(text):
    grid = defaultdict(lambda: None)
    charMap = {}
    for rowIdx,line in enumerate(text.strip().split("\n")):
        for colIdx,char in enumerate(line.strip()):
            if char is not WALL:
                coords = (colIdx, rowIdx)
                grid[coords] = char
                if char is not SPACE:
                    charMap[char] = coords
    return (grid, charMap)

def dfs(grid, pos):
    """
    Given a grid and (x,y) pos, returns all of the paths from
    pos that terminate in a non-WALL
    """
    seen = {pos}
    paths = []
    def search(pos, path):
        neighbors = [n for n in gridNeighbors(grid, pos) if n not in seen and grid[n] not in [WALL, None]]
        for n in neighbors:
            seen.add(n)
            if grid[n] == SPACE:
                search(n, path + [n])
            else:
                paths.append(path + [n])
    search(pos, [pos])
    return paths    


class Day18:
    def solve(self, text):
        (grid, charMap) = process(text)
        def _solve(grid, charMap):
            pos = charMap[ENTRANCE]
            paths = dfs(grid, pos)
            dists = []
            for path in paths:                
                start = path[0]
                end = path[-1]
                assert path[0] == pos
                assert grid[start] == ENTRANCE
                assert grid[end] != WALL
                if isKey(grid[end]):
#                     print("unlocking key {} at {} via path {}".format(grid[end], end, path))
                    key = grid[end]
                    lock = lockForKey(key)
                    assert key in charMap
                    # assert lock in charMap # The lock may not be in the map
                    assert charMap[key] == end
                    
                    # Make a copy of grid and charMap
                    _grid = grid.copy()
                    _charMap = charMap.copy()
                    # Move entrance, clear old entrance
                    _grid[start] = SPACE
                    _grid[end] = ENTRANCE
                    # Update the charmap, change entrance
                    _charMap[ENTRANCE] = end
                    # Update charmap, remove key
                    del _charMap[key]
                    
                    # If the lock exists, unlock it, remove from charMap and update grid
                    if lock in charMap:
                        # Unlock lock
                        _grid[_charMap[lock]] = SPACE
                        del _charMap[lock]
                    dists.append(len(path) - 1 + _solve(_grid, _charMap))
            if len(dists):
                return min(dists)
            else:
                return 0
        return _solve(grid, charMap)
        
        

text = """########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################"""
assert Day18().solve(text) == 86

text = """########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################"""
assert Day18().solve(text) == 132

text = """########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################"""

assert Day18().solve(text) == 81

text = """#################
#.........e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################"""
# Day18().solve(text) # This takes a long time

print(Day18().solve(readInput(18)))

# [Day 19](https://adventofcode.com/2019/day/19)

In [26]:
(STATIONARY, PULLED) = (0,1)
RESPONSES = [STATIONARY, PULLED]

class Day19:
    def solve(self, text):
        grid = {}
        for x in range(50):
            for y in range(50):
                # I might have done something wrong, but it seems like I can
                # only run the program 1 time per set of coordinates, so
                # this creates a new program for each coord to probe
                comp = IntCode(toIntList(text, sep=","), inputs=[x,y])
                resp = comp.runUntilOutput()
                assert resp in RESPONSES, "Expected {} in {}, x,y = {}".format(resp,RESPONSES,(x,y))
                grid[(x,y)] = resp
        return len([v for v in grid.values() if v == PULLED])

    def solveB(self, text):
        prog = toIntList(text,sep=",")
        def isPulled(x,y):
            return IntCode(prog,[x,y]).runUntilOutput() == PULLED
        x = y = 0
        while not isPulled(x+99,y):
            y += 1
            while not isPulled(x,y+99):
                x += 1
        return x*10000+y

assert Day19().solve(readInput(19)) == 154
assert Day19().solveB(readInput(19)) == 9791328

print(
    Day19().solve(readInput(19)),
    Day19().solveB(readInput(19))
)


154 9791328


In [27]:
def probe(xrange, yrange):
    prog = toIntList(readInput(19), sep=",")
    grid = {}
    for y in yrange:
        for x in xrange:
            comp = IntCode(prog, inputs=[x,y])
            resp = comp.runUntilOutput()
            assert resp in RESPONSES, "Expected {} in {}, x,y = {}".format(resp,RESPONSES,(x,y))
            grid[(x,y)] = resp
            assert resp in RESPONSES, "Expected {} in {}, x,y = {}".format(resp,RESPONSES,(x,y))
    out = ""
    for y in yrange:
        out += "{}: ".format(y)
        for x in xrange:
            out += {
                PULLED: '#',
                STATIONARY: '.'
            }[grid[(x,y)]]
        out += "\n"
    return out


## Used the probe to print out grid at various locations
# Tentative guesses:
(x,y) = (973,1444)
# print(probe(range(x-10,x+10+100), range(y-10,y+10+100)))

# tentative upper-right: x=1073, y=1444
# extrapolated lower-left: 973,1544
# extrapolated upper-left: 973,1444

In [28]:
# Attempt to figure out some patterns in the locations of the squares:
if False:
    g = Day19().solveB(readInput(19))
    out = ""
    for y in range(151):
        for x in range(151):
            if 11 <= x < 11+2 and 15 <= y < 15+2:
                assert g[(x,y)] == PULLED
                out += '2'            
            elif 22 <= x < 25 and 30 <= y < 33:
                assert g[(x,y)] == PULLED
                out += '3'
            elif 31 <= x < 35 and 42 <= y < 46:
                assert g[(x,y)] == PULLED
                out += '4'
            elif 42 <= x < 47 and 57 <= y < 62:
                assert g[(x,y)] == PULLED
                out += '5'
            elif 55 <= x < 55+6 and 74 <= y < 75+6:
                assert g[(x,y)] == PULLED
                out += '6'
            elif 66 <= x < 66+7 and 90 <= y < 90+7:
                assert g[(x,y)] == PULLED
                out += '7'
            elif 75 <= x < 75+8 and 102 <= y < 102+8:
                assert g[(x,y)] == PULLED
                out += '8'
            elif x == 103 and y == 150:
                assert g[(x,y)] == PULLED
                out += 'L' # Lower bound
            elif x == 121 and y == 150:
                assert g[(x,y)] == PULLED
                out += 'U' # Lower bound
            else:
                out += {
                    PULLED: '#',
                    STATIONARY: '.'
                }[g[(x,y)]]
        out += "\n"
    # print(out)

    # NW coords of nxn squares
    nw_coords = {
        2: (11,15),
        3: (22,30),
        4: (31,42),
        5: (42,57),
        6: (55,74),
        7: (66,90)
    }

    ne_coords = {
        2: (12,15),
        3: (24,30),
        4: (34,42),
        5: (46,57),
        6: (60,74),
        7: ()
    }

    # x and y changes from nxn square to (n+1)x(n+1) square
    dX = [11,9,11,13,11]
    dY = [15,12,15,17,16]

    # upper bounding line slope
    u1 = (0,0)
    u2 = (103,150)
    m_u = 150/103

    # lower bounding line  slope
    l1 = (0,0)
    l2 = (121,150)
    m_l = 150/121

In [29]:
# Attempt to use trigonometry to figure out the coords of the upper-left
if False:
    import math
    mUpper = 150/103
    mLower = 150/122

    def rad2deg(rad):
        return 180*rad/math.pi

    # Angle between lines
    aBetween = math.atan(abs((mUpper-mLower)/(1+mUpper*mLower)))
    print("aBetween deg {}".format(rad2deg(aBetween)))

    # Angle between upper and horizontal
    aUpper = math.atan(abs((mUpper-0)/(1+0)))
    print("aUpper deg {}".format(rad2deg(aUpper)))

    # Angle between lower and vertical
    aLower = (math.pi/2)-(aUpper+aBetween)
    print("aLower deg {}".format(rad2deg(aLower)))

    # Triangle where angle th is aBetween+aLower
    # And we want find the length of a
    # |---\                                       
    # | th -------\                               
    # |            --------\                      
    # a                     --------\             
    # |                              -------\     
    # ------------- =n= --------------------------

    # tan(th) = n / a
    # a = n / tan(th)

    theta = aBetween+aLower
    n = 100
    sideA = n / math.tan(theta)
    print("sideA {} ".format(sideA))

    # So a vertical line that slices the two bounding lines along the side of the left square will be n + sideA tall
    leftSliceLine = sideA + n
    print("vert slice line left {}".format(leftSliceLine))

    # Find the x-value where a line of height `leftSliceLine` intersects both the upper and lower lines
    # yUpper - yLower = 145
    # x*mUpper - x*mLower = 145
    # x = 145 / (mUpper-mLower)
    upperLeftX = leftSliceLine / (mUpper-mLower)
    print("upperLeftX {}".format(upperLeftX))

    # upperLeft Y = the upper boundary Y value minus sideA
    upperLeftY = (upperLeftX*mUpper) - sideA
    print("upperLeftY {}".format(upperLeftY))

    # round away to closest whole number
    coords = (math.floor(upperLeftX), math.floor(upperLeftY))
    print("Coords {}".format(coords))

    (x,y) = coords
    x*10000+y

In [5]:
ex0 = """#########
#b.A.@.a#
#########""".split("\n")

# 86
ex01 = """########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################""".split("\n")

# 132
ex = """########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################""".split("\n")

# 136
ex2 = """#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################""".split("\n")

# 81
ex4 = """########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################""".split("\n")

In [12]:
def Day18(data=get_input(18,2019)):
  WALL = '#'
  OPEN = '.'
  # data = ex2
  def parse(data):
    G = defaultdict(lambda:None)
    DOORS = {}
    KEYS = {}
    for y,line in enumerate(data):
      for x,c in enumerate(line):
        G[(x,y)] = c
        if c not in [WALL,OPEN]:
          if c == '@':
            G[(x,y)] = '.'
            loc = (x,y)
          elif c.isupper():
            DOORS[c] = (x,y)
          else:
            KEYS[c] = (x,y)
    return G,DOORS,KEYS,loc

  def solve():
    G,DOORS,KEYS,cur = parse(data)
    keycount = len(KEYS)
    nodes = [(0, keycount, cur, "")]
    seen = set()
    while nodes:
      dist,cost,node,keys = heapq.heappop(nodes)
      if len(keys) == keycount:
        return dist
      if (node,keys) in seen:
        continue
      else:
        seen.add( (node, keys) )
      for n in aoc_utils.neighbors4(node):
        if n not in G:
          continue
        if G[n] == WALL:
          continue
        if G[n].isupper() and G[n].lower() not in keys:
          continue
        next_dist = dist + 1
        next_cost = cost
        next_keys = keys
        if G[n].islower() and G[n] not in keys:
          next_cost -= 1
          next_keys = cat(sorted(keys + G[n]))
        heapq.heappush(nodes, (next_dist, next_cost, n, next_keys) )

  return solve()
Day18()

4620

# [Day 22](https://adventofcode.com/2019/day/22)

In [30]:
def cut(cards, n):
    return cards[n:] + cards[:n]

assert cut(list(range(10)), 3) == [3,4,5,6,7,8,9,0,1,2]
assert cut(list(range(10)), -4) == [6,7,8,9,0,1,2,3,4,5]

def deal(cards, inc):
    out = [None] * len(cards)
    assert len(out) == len(cards)
    for idx,card in enumerate(cards):
        out[idx*inc%len(out)] = card
    return out

def dealIntoNew(cards):
    return list(reversed(cards))

assert deal(list(range(10)), 3) == [0, 7, 4, 1, 8, 5, 2, 9, 6, 3]

def parseInstruction(instr):
    if instr.startswith("deal into"):
        return lambda cards: dealIntoNew(cards)
    else:
        count = int(instr.split(' ')[-1])
        if instr.startswith("deal with"):
            return lambda cards: deal(cards, count)
        elif instr.startswith("cut"):
            return lambda cards: cut(cards, count)
    assert False
    
def shuffle(cards, instructions):
    for line in instructions.strip().split('\n'):
        instr = parseInstruction(line)
        cards = instr(cards)
    return cards
        
assert shuffle(list(range(10)), """
deal with increment 7
deal into new stack
deal into new stack
""") == [0, 3, 6, 9, 2, 5, 8, 1, 4, 7]

assert shuffle(list(range(10)), """
cut 6
deal with increment 7
deal into new stack
""") == [3, 0, 7, 4, 1, 8, 5, 2, 9, 6]

assert shuffle(list(range(10)), """
deal with increment 7
deal with increment 9
cut -2
""") == [6, 3, 0, 7, 4, 1, 8, 5, 2, 9]

assert shuffle(list(range(10)), """
deal into new stack
cut -2
deal with increment 7
cut 8
cut -4
deal with increment 7
cut 3
deal with increment 9
deal with increment 3
cut -1
""") == [9, 2, 5, 8, 1, 4, 7, 0, 3, 6]


class Day22:
    def solve(self, text):
        return shuffle(list(range(10007)), text).index(2019)

assert Day22().solve(readInput(22)) == 6289

print(
    Day22().solve(readInput(22))
)

6289
